In [ ]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

from scipy.optimize import minimize

# General imports
import os
import json
import numpy as np
from datetime import datetime

# runtime imports
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import RealAmplitudes
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

aer_sim = AerSimulator(method='statevector')
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=3)

# custom module
from susy_qm import calculate_wz_hamiltonian

In [5]:
# Parameters
N = 2
a = 1.0
c = 0
potential = "linear"#'quadratic'
boundary_condition = 'dirichlet'#'periodic'

In [ ]:
# Create cost function
cost_history_dict = {
"prev_vector": None,
"iters": 0,
"cost_history": [],
}


def cost_func(params, ansatz, hamiltonian, estimator):
   
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)

    return energy

In [9]:
cut_offs_list = [4]
shots = None

starttime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

if shots != None:
    folder = 'Noise\\N'+str(N)
else:
    folder = 'No Noise\\N'+str(N)

#Create directory for files
base_path = r"C:\Users\Johnk\OneDrive\Desktop\PhD 2024\Quantum Computing Code\Quantum-Computing\SUSY\Wess-Zumino\VQE\COBYLA\Files\{}\\{}\\".format(potential, folder)
os.makedirs(base_path, exist_ok=True)

print(f"Running for {potential} potential")

for cut_off in cut_offs_list:

    print(f"Running for cutoff: {cut_off}")

    #calculate Hamiltonian and expected eigenvalues
    H = calculate_wz_hamiltonian(cut_off, N, a, potential, boundary_condition, c)
    
    num_eigenvalues = 6
    eigenvalues = np.sort(np.linalg.eig(H)[0][:12])
    min_eigenvalue = min(eigenvalues.real)

    #create qiskit Hamiltonian Pauli string
    hamiltonian = SparsePauliOp.from_operator(H)
    num_qubits = hamiltonian.num_qubits
    print("Num qubits: ", num_qubits)

    ansatz = RealAmplitudes(num_qubits=num_qubits, reps=1)
    ansatz_isa = pm.run(ansatz)
    hamiltonian_isa = hamiltonian.apply_layout(layout=ansatz_isa.layout)

    num_params = ansatz.num_parameters
    x0_str = '0.25 * np.pi * np.random.random(num_params)'

    
    # VQE
    vqe_start = datetime.now()

    num_vqe_runs = 10
    max_iterations = 10000
    tolerance = 1e-6


    energies = []
    x_values = []
    success = []
    run_times = []
    num_iters = []

    backend=aer_sim
    estimator = Estimator(mode=backend)

    for i in range(num_vqe_runs):

        x0 = 2 * np.pi * np.random.random(num_params)
        run_start = datetime.now()

        if i % 10 == 0:
            print(f"Run: {i}")

        res = minimize(
            cost_func,
            x0,
            method= "COBYLA",
            args= (ansatz_isa, hamiltonian_isa, estimator),
            options= {'maxiter':max_iterations, 'tol':tolerance}
        )
        energies.append(res.fun)
        x_values.append(res.x)
        success.append(res.success)
        num_iters.append(res.nfev)

        run_end = datetime.now()
        run_time = run_end - run_start
        run_times.append(run_time)
        

    vqe_end = datetime.now()
    vqe_time = vqe_end - vqe_start

Running for linear potential
Running for cutoff: 4
Num qubits:  6
Run: 0


In [10]:
#Save run
run = {
    'starttime': starttime,
    'potential': potential,
    'boundary_condition': boundary_condition,
    'c': c,
    'cutoff': cut_off,
    'num_sites': N,
    'exact_eigenvalues': [x.real.tolist() for x in eigenvalues],
    'ansatz': 'RealAmplitudes',
    'x0': x0_str,
    'num_VQE': num_vqe_runs,
    'backend': 'aer_simulator',
    'shots': shots,
    'min_function': {'name': 'minimizer',
                        'method': "COBYLA",
                        'maxiter':max_iterations,
                        'tolerance': tolerance
                        },
    'results': energies,
    'params': [x.tolist() for x in x_values],
    'num_iters': num_iters,
    'success': np.array(success, dtype=bool).tolist(),
    'run_times': [str(x) for x in run_times],
    'total_run_time': str(vqe_time)
}

# Save the variable to a JSON file
path = base_path + "{}_{}.json".format(potential, cut_off)
with open(path, 'w') as json_file:
    json.dump(run, json_file, indent=4)

In [ ]:


#base_path = r"C:\Users\Johnk\OneDrive\Desktop\PhD 2024\Quantum Computing Code\Quantum-Computing\SUSY\PennyLane\SUSY VQE\Shot Noise\Files\{}\\{}\\"
#create_vqe_plots(potential=potential, base_path=base_path, folder=folder, cut_off_list=cut_offs_list)